# **Основи бібліотеки Bokeh**

**Ваша перша діаграма**

In [ ]:
# my_first_plot.py
from bokeh.plotting import figure, output_file, show

output_file('my_first_graph.html')

x = [1, 3, 5, 7]
y = [2, 4, 6, 8]

p = figure()

p.scatter(x, y, size=10, color='red', legend_label='circle')
p.line(x, y, color='blue', legend_label='line')
p.scatter(y, x, color='gold', size=10, legend_label='triangle')

p.legend.click_policy = 'hide'

show(p)

# **Bokeh та Pandas: вивчення набору даних “Масовані ракетні удари по Україні”**


**Завантаження даних у Pandas**

In [ ]:
#loading_data.py
import pandas as pd

df = pd.read_csv('missile_attacks_daily.csv', sep=';')
print(df)

     time_start    time_end               model    launch_place  \
0    2023-06-08  2023-06-08              Kalibr       Black Sea   
1    2024-11-11  2024-11-12           X-59/X-69    Kursk oblast   
2    2024-10-15  2024-10-16                X-59    Kursk oblast   
3    2024-09-12  2024-09-12          Iskander-M    Kursk oblast   
4    2024-09-11  2024-09-11           X-59/X-69  Bryansk oblast   
..          ...         ...                 ...             ...   
614  2023-12-11  2023-12-11                X-59             NaN   
615  2023-12-05  2023-12-05  Reconnaissance UAV             NaN   
616  2023-11-21  2023-11-22                X-22      south-east   
617  2023-08-27  2023-08-27      Shahed-136/131             NaN   
618  2023-08-12  2023-08-12      Shahed-136/131      south-east   

                  target  launched  destroyed  not_reach_goal  
0        Cherkasy oblast       2.0          0             NaN  
1       Chernihiv oblast       2.0          0             NaN  
2  

**The Bokeh ColumnDataSource**

In [ ]:
# column_datasource.py
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool

output_file('columndatasource_example.html')

df = pd.read_csv('missile_attacks_daily.csv', sep=';')

sample = df.sample(50)
source = ColumnDataSource(sample)

p = figure()
p.scatter(x='launched', y='destroyed',
         source=source,
         size=10, color='green')

hover = HoverTool()
hover.tooltips=[
    ('Дата початку атаки', '@time_start'),
    ('Кількість запущених ракет чи БПЛА', '@launched'),
    ('Кількість знищених ракет чи БПЛА', '@destroyed'),
    ('Тип ракет чи БПЛА', '@model')
]

p.add_tools(hover)

show(p)

# **Категорійні дані та стовпчикові діаграми: атаки за містами чи областями**

In [ ]:
#launched_on_target.py
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool

from bokeh.palettes import turbo
from bokeh.transform import factor_cmap
output_file('launched_on_target.html')

df = pd.read_csv('missile_attacks_daily.csv', sep=';')

grouped = df.groupby('target')[['launched', 'destroyed', 'not_reach_goal']].sum()

# print(grouped)

source = ColumnDataSource(grouped)
location = source.data['target'].tolist()
p = figure(x_range=location)

color_map = factor_cmap(field_name='target',
                    palette=turbo(29), factors=location)

p.vbar(x='target', top='launched', source=source, width=0.70, color=color_map)

p.title.text = 'Кількість атак відповідно до цілей'
p.xaxis.axis_label = 'Місто чи область'
p.yaxis.axis_label = 'Кількість запущених ракет чи БПЛА'
p.xaxis.major_label_orientation = 'vertical'

hover = HoverTool()
hover.tooltips = [
    ('З них', '@destroyed було знищено / @not_reach_goal не досягли цілі')]

hover.mode = 'vline'

p.add_tools(hover)

show(p)

# **Стовпчикові діаграми з накопиченням та дані підвибірки: атаки за областями**

In [ ]:
#attack_results.py
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral3

output_file('attack_results’.html')

df = pd.read_csv('missile_attacks_daily.csv', sep=';')

filter = df['target'].isin(('Kyiv oblast', 'Odesa oblast'))
df = df[filter]

# print(df.shape)

grouped = df.groupby('target')[['launched', 'destroyed', 'not_reach_goal']].sum()

source = ColumnDataSource(grouped)
location = source.data['target'].tolist()
p = figure(x_range=location)

p.vbar_stack(stackers=['launched', 'destroyed', 'not_reach_goal'],
             x='target', source=source,
             legend_label = ['Запущені', 'Знищені', 'Не досягли цілі'],
             width=0.5, color=Spectral3)

p.title.text ='Результати атак'
p.legend.location = 'bottom_left'

p.xaxis.axis_label = 'Цілі'
p.xgrid.grid_line_color = None	#remove the x grid lines

p.yaxis.axis_label = 'Кількість ракет чи БПЛА'

show(p)

# **Часові ряди та анотації: атаки у часі**

In [ ]:
#my_first_timeseries.py
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral3
output_file('simple_timeseries_plot.html')

df = pd.read_csv('missile_attacks_daily.csv', sep=';')

# Переконуємося, що time_start має формат datetime
df['time_start'] = pd.to_datetime(df['time_start'], format='ISO8601')

grouped = df.groupby('time_start')[['launched', 'destroyed', 'not_reach_goal']].sum()

print(grouped)
source = ColumnDataSource(grouped)

p = figure(x_axis_type='datetime')

p.line(x='time_start', y='launched', line_width=2, source=source, legend_label='Запущені')
p.line(x='time_start', y='destroyed', line_width=2, source=source, color=Spectral3[1], legend_label='Знищені')
p.line(x='time_start', y='not_reach_goal', line_width=2, source=source, color=Spectral3[2], legend_label='Не досягли цілі')

p.yaxis.axis_label = 'Кількість ракет чи БПЛА'

show(p)

**Повторна вибірка даних часових рядів**

In [ ]:
#my_first_timeseries.py
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import TolRainbow3
output_file('simple_timeseries_plot.html')

df = pd.read_csv('missile_attacks_daily.csv', sep=';')

#make sure time_start is a datetime format
df['time_start'] = pd.to_datetime(df['time_start'], format='ISO8601')

# grouped = df.groupby('time_start')[['launched', 'destroyed', 'not_reach_goal']].sum()
grouped = df.groupby(pd.Grouper(key='time_start', freq='M'))[['launched', 'destroyed', 'not_reach_goal']].sum()

print(grouped)
source = ColumnDataSource(grouped)

p = figure(x_axis_type='datetime')

p.line(x='time_start', y='launched', line_width=2, source=source, color=TolRainbow3[0], legend_label='Запущені')
p.line(x='time_start', y='destroyed', line_width=2, source=source, color=TolRainbow3[1], legend_label='Знищені')
p.line(x='time_start', y='not_reach_goal', line_width=2, source=source, color=TolRainbow3[2], legend_label='Не досягли цілі')
p.legend.location = 'top_left'
p.yaxis.axis_label = 'Кількість ракет чи БПЛА'

show(p)

**Анотування тенденцій у сюжетах**

In [ ]:
# annotating_trends.py
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from datetime import datetime
from bokeh.palettes import TolRainbow3
output_file('annotating_trends.html')

df = pd.read_csv('missile_attacks_daily.csv', sep=';')

#filter for Kyiv oblast and Odesa oblast
filter = df['target'].isin(('Kyiv oblast', 'Odesa oblast'))
df = df[filter]

df['time_start'] = pd.to_datetime(df['time_start'], format='ISO8601')
group = df.groupby(pd.Grouper(key='time_start', freq='M'))[['launched', 'destroyed', 'not_reach_goal']].sum()

source = ColumnDataSource(group)

p = figure(x_axis_type="datetime")

p.line(x='time_start', y='launched', line_width=2, source=source, color=TolRainbow3[0], legend_label='Запущені')
p.line(x='time_start', y='destroyed', line_width=2, source=source, color=TolRainbow3[1], legend_label='Знищені')
p.line(x='time_start', y='not_reach_goal', line_width=2, source=source, color=TolRainbow3[2], legend_label='Не досягли цілі')
p.legend.location = 'top_left'

p.title.text = 'Атаки по Київській та Одеській областях'

p.yaxis.axis_label = 'Кількість ракет чи БПЛА'

show(p)

**Атаки по Київській області**

In [ ]:
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from datetime import datetime
from bokeh.palettes import TolRainbow3
output_file('annotating_trends_1.html')

df = pd.read_csv('missile_attacks_daily.csv', sep=';')

#filter for Kyiv oblast
filter = df['target']=='Kyiv oblast'
df = df[filter]

df['time_start'] = pd.to_datetime(df['time_start'], format='ISO8601')
group = df.groupby(pd.Grouper(key='time_start', freq='M'))[['launched', 'destroyed', 'not_reach_goal']].sum()

source = ColumnDataSource(group)

p = figure(x_axis_type="datetime")

p.line(x='time_start', y='launched', line_width=2, source=source, color=TolRainbow3[0], legend_label='Запущені')
p.line(x='time_start', y='destroyed', line_width=2, source=source, color=TolRainbow3[1], legend_label='Знищені')
p.line(x='time_start', y='not_reach_goal', line_width=2, source=source, color=TolRainbow3[2], legend_label='Не досягли цілі')
p.legend.location = 'top_left'

p.title.text = 'Атаки по Київській області'

p.yaxis.axis_label = 'Кількість ракет чи БПЛА'

show(p)

**Атаки по Одеській області**

In [ ]:
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from datetime import datetime
from bokeh.palettes import TolRainbow3
output_file('annotating_trends_2.html')

df = pd.read_csv('missile_attacks_daily.csv', sep=';')

#filter for Odesa oblast
filter = df['target']=='Odesa oblast'
df = df[filter]

df['time_start'] = pd.to_datetime(df['time_start'], format='ISO8601')
group = df.groupby(pd.Grouper(key='time_start', freq='M'))[['launched', 'destroyed', 'not_reach_goal']].sum()

source = ColumnDataSource(group)

p = figure(x_axis_type="datetime")

p.line(x='time_start', y='launched', line_width=2, source=source, color=TolRainbow3[0], legend_label='Запущені')
p.line(x='time_start', y='destroyed', line_width=2, source=source, color=TolRainbow3[1], legend_label='Знищені')
p.line(x='time_start', y='not_reach_goal', line_width=2, source=source, color=TolRainbow3[2], legend_label='Не досягли цілі')
p.legend.location = 'top_left'

p.title.text = 'Атаки по Одеській області'

p.yaxis.axis_label = 'Кількість ракет чи БПЛА'

show(p)

**Атаки по Київській області із доданою анотацією**

In [ ]:
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from datetime import datetime
from bokeh.palettes import TolRainbow3
from bokeh.models import BoxAnnotation
output_file('annotating_trends_1_1.html')

df = pd.read_csv('missile_attacks_daily.csv', sep=';')

#filter for Kyiv oblast
filter = df['target']=='Kyiv oblast'
df = df[filter]

df['time_start'] = pd.to_datetime(df['time_start'], format='ISO8601')
group = df.groupby(pd.Grouper(key='time_start', freq='M'))[['launched', 'destroyed', 'not_reach_goal']].sum()

source = ColumnDataSource(group)

p = figure(x_axis_type="datetime")

p.line(x='time_start', y='launched', line_width=2, source=source, color=TolRainbow3[0], legend_label='Запущені')
p.line(x='time_start', y='destroyed', line_width=2, source=source, color=TolRainbow3[1], legend_label='Знищені')
p.line(x='time_start', y='not_reach_goal', line_width=2, source=source, color=TolRainbow3[2], legend_label='Не досягли цілі')
p.legend.location = 'top_left'

p.title.text = 'Атаки по Київській області'

p.yaxis.axis_label = 'Кількість ракет чи БПЛА'

box_left = pd.to_datetime('2024-7-1')
box_right = pd.to_datetime('2024-9-1')

box = BoxAnnotation(left=box_left, right=box_right,
                    line_width=1, line_color='black', line_dash='dashed',
                    fill_alpha=0.2, fill_color='orange')

p.add_layout(box)

show(p)

**Атаки по Одеській області із доданою анотацією**

In [ ]:
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from datetime import datetime
from bokeh.palettes import TolRainbow3
from bokeh.models import BoxAnnotation
output_file('annotating_trends_2_2.html')

df = pd.read_csv('missile_attacks_daily.csv', sep=';')

#filter for Odesa oblast
filter = df['target']=='Odesa oblast'
df = df[filter]

df['time_start'] = pd.to_datetime(df['time_start'], format='ISO8601')
group = df.groupby(pd.Grouper(key='time_start', freq='M'))[['launched', 'destroyed', 'not_reach_goal']].sum()

source = ColumnDataSource(group)

p = figure(x_axis_type="datetime")

p.line(x='time_start', y='launched', line_width=2, source=source, color=TolRainbow3[0], legend_label='Запущені')
p.line(x='time_start', y='destroyed', line_width=2, source=source, color=TolRainbow3[1], legend_label='Знищені')
p.line(x='time_start', y='not_reach_goal', line_width=2, source=source, color=TolRainbow3[2], legend_label='Не досягли цілі')
p.legend.location = 'top_left'

p.title.text = 'Атаки по Одеській області'

p.yaxis.axis_label = 'Кількість ракет чи БПЛА'

box_left = pd.to_datetime('2024-10-1')
box_right = pd.to_datetime('2024-12-1')

box = BoxAnnotation(left=box_left, right=box_right,
                    line_width=1, line_color='black', line_dash='dashed',
                    fill_alpha=0.2, fill_color='orange')

p.add_layout(box)

show(p)

# **Просторова база даних: відображення цілей атак**

In [1]:
#Інсталяція бібліотеки geopy
pip install geopy==2.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: geographiclib
    Found existing installation: geographiclib 2.0
    Uninstalling geographiclib-2.0:
      Successfully uninstalled geographiclib-2.0
  Attempting uninstall: geopy
    Found existing installation: geopy 2.4.1
    Uninstalling geopy-2.4.1:
      Successfully uninstalled geopy-2.4.1


In [ ]:
import pandas as pd
import time
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

def main():
    df = pd.read_csv('missile_attacks_daily_1.csv', index_col=None, header=0, sep=";")

    geolocator = Nominatim(user_agent="your_email@example.com", timeout=10)

    def get_location(place):
        try:
            location = geolocator.geocode(place)
            time.sleep(1)
            return location
        except GeocoderTimedOut:
            return get_location(place)

    df['location'] = df['target'].apply(get_location)
    df['latitude'] = df['location'].apply(lambda loc: loc.latitude if loc else None)
    df['longitude'] = df['location'].apply(lambda loc: loc.longitude if loc else None)

    df.drop(columns=['location'], inplace=True)

    df.to_csv('geocoding-output.csv', index=False)

if __name__ == '__main__':
    main()

In [ ]:
# target_locations.py
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, Range1d
import numpy as np
output_file('mapping_targets.html')

df = pd.read_csv('geocoding-output.csv', sep=',')

k = 6378137
df['MercatorX'] = df['longitude'] * (k * np.pi/180.0)
df['MercatorY'] = np.log(np.tan((90 + df['latitude']) * np.pi/360.0)) * k

grouped = df.groupby(['MercatorX', 'MercatorY'])['launched'].sum().reset_index()

filter = grouped['launched'] != 0
grouped = grouped[filter]

source = ColumnDataSource(grouped)

left = -1000000
right = 5000000
bottom = 2500000
top = 5500000

p = figure(x_range=Range1d(left, right), y_range=Range1d(bottom, top))

p.add_tile("CartoDB Positron", retina=True)

p.scatter(x='MercatorX', y='MercatorY', size=5, source=source, line_color='grey', fill_color='yellow')

p.axis.visible = False

show(p)